In [16]:
image_file = "/home/nemo/code/data/robothon_data/images/frame101.jpg"


In [18]:
import cv2 as cv
from math import atan2, cos, sin, sqrt, pi
import numpy as np
 
def drawAxis(img, p_, q_, color, scale):
    p = list(p_)
    q = list(q_)

    ## [visualization1]
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))

    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)

    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)

    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv.LINE_AA)
## [visualization1]
 
def getOrientation(pts, img):
    ## [pca]
    # Construct a buffer used by the pca analysis
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]

    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)

    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    ## [pca]

    ## [visualization]
    # Draw the principal components
    cv.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (255, 255, 0), 1)
    drawAxis(img, cntr, p2, (0, 0, 255), 5)

    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    ## [visualization]

    # Label with the rotation angle
    label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
    textbox = cv.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
    cv.putText(img, label, (cntr[0], cntr[1]), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA)

    return angle
 
# Load the image
img = cv.imread(image_file)
 
# Was the image there?
if img is None:
    print("Error: File not found")
    exit(0)
 
cv.imshow('Input Image', img)
 
# Convert image to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
# Convert image to binary
_, bw = cv.threshold(gray, 50, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
 
# Find all the contours in the thresholded image
contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)
 
for i, c in enumerate(contours):

    # Calculate the area of each contour
    area = cv.contourArea(c)

    # Ignore contours that are too small or too large
    if area < 3700 or 100000 < area:
        continue

    # Draw each contour only for visualisation purposes
    cv.drawContours(img, contours, i, (0, 0, 255), 2)

    # Find the orientation of each shape
    getOrientation(c, img)
 
cv.imshow('Output Image', img)
cv.waitKey(0)
cv.destroyAllWindows()
  
# Save the output image to the current directory
cv.imwrite("output_img.jpg", img)

True

In [1]:
# import numpy as np
# import cv2

# img = cv2.imread("/home/nemo/code/data/robothon_data/images/frame101.jpg")
# imgGry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# ret , thrash = cv2.threshold(imgGry, 240 , 255, cv2.CHAIN_APPROX_NONE)
# contours , hierarchy = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)



# for contour in contours:
#     approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
#     cv2.drawContours(img, [approx], 0, (0, 0, 0), 5)
#     x = approx.ravel()[0]
#     y = approx.ravel()[1] - 5
#     if len(approx) == 3:
#         cv2.putText( img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0) )
#     elif len(approx) == 4 :
#         x, y , w, h = cv2.boundingRect(approx)
#         aspectRatio = float(w)/h
#         print(aspectRatio)
#         if aspectRatio >= 0.95 and aspectRatio < 1.05:
#             cv2.putText(img, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

#         else:
#             cv2.putText(img, "rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

#     elif len(approx) == 5 :
#         cv2.putText(img, "pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
#     elif len(approx) == 10 :
#         cv2.putText(img, "star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
#     else:
#         cv2.putText(img, "circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

# cv2.imshow('shapes', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6666666666666666
1.0
0.75
1.5
1.0
0.4
0.6666666666666666
1.5
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.5
1.0
1.0
0.6666666666666666
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [10]:
from pprint import PrettyPrinter as pp

In [11]:
import cv2
import numpy as np

img = cv2.imread("/home/nemo/code/data/robothon_data/images/frame101.jpg")

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# cnts = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# for cnt in cnts:
#     approx = cv2.contourArea(cnt)
#     print(approx)

    
    
#     ##################
    
    
# import cv2

font = cv2.FONT_HERSHEY_SIMPLEX    

# frame = cv2.imread('text.jpg')
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# blur = cv2.GaussianBlur(gray, (7, 7), 0.5)
# edge = cv2.Canny(blur, 0, 50, 3)

contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)


    
    
pp(contours)
# print(hierarchy)
    
    
# for contour, hier in zip(contours, hierarchy):
#     (x,y,w,h) = cv2.boundingRect(contour)
#     cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     cv2.putText(img, ('width = {}, height = {}'.format(w, h)),
#                 (x+30, y+30),
#                 font,
#                 1,
#                 (0, 255, 0),
#                 2,
#                 cv2.LINE_AA)
    
cv2.imshow('image', img)
cv2.imshow('Binary',thresh_img)
cv2.waitKey()
cv2.destroyAllWindows()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [ ]:
p -> (a,b)

def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [54]:
import cv2
import numpy as np
import math


def verify_buttons(hough_circles):
    if len(hough_circles[0]) == 2:
        # extract the 2 points from the hough object (list)
        c1 = [hough_circles[0][0][0], hough_circles[0][0][1]]
        c2 = [hough_circles[0][1][0], hough_circles[0][1][1]]
        dist_circles = math.dist(c1, c2)
        
        print(dist_circles)
        
        if 18 < int(dist_circles) < 22:
            print(hough_circles[0])
            return hough_circles
        else:
            print("min distance criteria of circles broke")
            return None
    else:
        print("None or more than 2 keypoints detected; discarding values...")
        return None
    
    
def get_points(hough_circles):
    c1 = (hough_circles[0][0][0], hough_circles[0][0][1])
    c2 = (hough_circles[0][1][0], hough_circles[0][1][1])
    
    return [c1, c2]


def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

def get_angle(points):
    point_1 = points[1]
    point_2 = points[0]
    theta = angle_between(point_1, point_2)
#     delY = point_1[1] - point_2[1]
#     delX = point_1[0] - point_2[0]
#     theta_rad = math.atan2(delY, delX)
#     theta_deg = theta_rad*(180/np.pi)
    return theta  
    
    
def detect_buttons(img):
# Convert to grayscale.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur using 3 * 3 kernel.
#     gray_blurred = cv2.blur(gray, (3, 3))
    gray_blurred = gray

    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 10, 
                                        param1 = 50, param2 = 30, minRadius = 9, maxRadius = 15)
    
    
    # verifies if the 2 buttons are detected using heuristics
    verified_circles = verify_buttons(detected_circles)
    
    if verified_circles is not None:
        
        circle_centers = get_points(verified_circles)
        angle = get_angle(circle_centers)
        print("angle: ", angle)
        
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]

            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), r, (0, 255, 0), 2)

            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
            
    return img



# def get_angle(points):
#     if len(points) == 2:
# #         delY = points[1][1] - points[0][1]
# #         delX = points[1][0] - points[0][0]
#         delY = points[0][1] - points[1][1]
#         delX = points[0][0] - points[1][0]
#         # slope = delY/delX
#         theta = math.atan2(delY, delX)
#         theta_deg = theta*(180/np.pi)
#         return theta_deg
    
    

    
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

# image_path = "/home/nemo/code/repos/robothon/code_repos/notebooks/zero_image.png"



img = cv2.imread(image_path, cv2.IMREAD_COLOR)

image = detect_buttons(img)

cv2.imshow("Detected Circle", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

21.213203435596427
[[428.5 237.5  13.2]
 [449.5 240.5   9.6]]
angle:  359.1507086653912


In [55]:
import cv2
import numpy as np
import math


def verify_buttons(hough_circles):
    if len(hough_circles[0]) == 2:
        # extract the 2 points from the hough object (list)
        c1 = [hough_circles[0][0][0], hough_circles[0][0][1]]
        c2 = [hough_circles[0][1][0], hough_circles[0][1][1]]
        dist_circles = math.dist(c1, c2)
        
        print(dist_circles)
        
        if 18 < int(dist_circles) < 22:
            print(hough_circles[0])
            return hough_circles
        else:
            print("min distance criteria of circles broke")
            return None
    else:
        print("None or more than 2 keypoints detected; discarding values...")
        return None
    
    
def get_points(hough_circles):
    c1 = [hough_circles[0][0][0], hough_circles[0][0][1]]
    c2 = [hough_circles[0][1][0], hough_circles[0][1][1]]
    
    return [c1, c2]

def get_angle(points):
    point_1 = points[1]
    point_2 = points[0]
    delY = point_1[1] - point_2[1]
    delX = point_1[0] - point_2[0]
    theta_rad = math.atan2(delY, delX)
    theta_deg = theta_rad*(180/np.pi)
    return theta_deg    
    
    
def detect_buttons(img):
# Convert to grayscale.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur using 3 * 3 kernel.
#     gray_blurred = cv2.blur(gray, (3, 3))
    gray_blurred = gray

    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 10, 
                                        param1 = 50, param2 = 30, minRadius = 9, maxRadius = 15)
    
    
    # verifies if the 2 buttons are detected using heuristics
    verified_circles = verify_buttons(detected_circles)
    
    if verified_circles is not None:
        
        circle_centers = get_points(verified_circles)
        angle = get_angle(circle_centers)
        print("angle: ", angle)
        
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]

            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), r, (0, 255, 0), 2)

            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
            
    return img



def get_angle(points):
    if len(points) == 2:
        delY = points[1][1] - points[0][1]
        delX = points[1][0] - points[0][0]
        # slope = delY/delX
        theta = math.atan2(delY, delX)
        theta_deg = theta*(180/np.pi)
        return theta_deg
    
    

    
# images_dir = "/home/nemo/code/data/robothon_data/images/"
# image_name = "frame101.jpg"

# image_path = images_dir + image_name

image_path = "/home/nemo/code/repos/robothon/code_repos/notebooks/zero_image.png"



img = cv2.imread(image_path, cv2.IMREAD_COLOR)

image = detect_buttons(img)

cv2.imshow("Detected Circle", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

20.024984394500787
[[362.5 215.5   9.6]
 [382.5 214.5   9.6]]
angle:  -2.8624052261117474


In [53]:
import cv2
import numpy as np
import math


def verify_buttons(hough_circles):
    if len(hough_circles[0]) == 2:
        # extract the 2 points from the hough object (list)
        c1 = [hough_circles[0][0][0], hough_circles[0][0][1]]
        c2 = [hough_circles[0][1][0], hough_circles[0][1][1]]
        dist_circles = math.dist(c1, c2)
        
        print(dist_circles)
        
        if 18 < int(dist_circles) < 22:
            print(hough_circles[0])
            return hough_circles
        else:
            print("min distance criteria of circles broke")
            return None
    else:
        print("None or more than 2 keypoints detected; discarding values...")
        return None
    
    
def get_points(hough_circles):
    c1 = [hough_circles[0][0][0], hough_circles[0][0][1]]
    c2 = [hough_circles[0][1][0], hough_circles[0][1][1]]
    
    return [c1, c2]

def get_angle(points):
    point_1 = points[0]
    point_2 = points[1]
    delY = point_1[1] - point_2[1]
    delX = point_1[0] - point_2[0]
    theta_rad = math.atan2(delY, delX)
    theta_deg = theta_rad*(180/np.pi)
    return theta_deg    
    
    
def detect_buttons(img):
# Convert to grayscale.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur using 3 * 3 kernel.
#     gray_blurred = cv2.blur(gray, (3, 3))
    gray_blurred = gray

    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 10, 
                                        param1 = 50, param2 = 30, minRadius = 9, maxRadius = 15)
    
    
    # verifies if the 2 buttons are detected using heuristics
    verified_circles = verify_buttons(detected_circles)
    
    if verified_circles is not None:
        
        circle_centers = get_points(verified_circles)
        angle = get_angle(circle_centers)
        print("angle: ", angle)
        
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]

            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), r, (0, 255, 0), 2)

            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
            
    return img



def get_angle(points):
    if len(points) == 2:
        delY = points[1][1] - points[0][1]
        delX = points[1][0] - points[0][0]
        # slope = delY/delX
        theta = math.atan2(delY, delX)
        theta_deg = theta*(180/np.pi)
        return theta_deg
    
    

    
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

# image_path = "/home/nemo/code/repos/robothon/code_repos/notebooks/zero_image.png"



img = cv2.imread(image_path, cv2.IMREAD_COLOR)

image = detect_buttons(img)

cv2.imshow("Detected Circle", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

21.213203435596427
[[428.5 237.5  13.2]
 [449.5 240.5   9.6]]
angle:  8.13010235415598


In [ ]:
#import the necessary modules
import freenect
import cv2
import numpy as np
import time
import math





def get_blob_params():
    params = cv2.SimpleBlobDetector_Params()

    params.filterByArea = True
    params.minArea = 150
    # params.maxArea = 250

    params.filterByCircularity = True
    params.minCircularity = 0.85
    
    return params


def get_button_centers(image):
    
    params = get_blob_params()

    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(image)
    if len(keypoints)==2:

        print(keypoints)

        print("---")
        print(get_keypoint_centers(keypoints))
        print("--------")
        print(get_angle(get_keypoint_centers(keypoints)))

        blank = np.zeros((1, 1))
        blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
        return blobs
    else:
        pass


def show_image(image, image_label):
    cv2.imshow(image_label, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def get_keypoint_centers(keypoints):
#     points_arr = []
#     for point in keypoints:
#         points_arr.append(point.pt)
    points_arr = [point.pt for point in keypoints]
    return points_arr
        
        
def get_angle(points):
    if len(points) == 2:
        delY = points[1][1] - points[0][1]
        delX = points[1][0] - points[0][0]
        # slope = delY/delX
        theta = math.atan2(delY, delX)
        theta_deg = theta*(180/np.pi)
        return theta_deg


# def verify_points():
    






#function to get RGB image from kinect
def get_video():
    array, success = freenect.sync_get_video()
    # convert to BGR
    array = cv2.cvtColor(array,cv2.COLOR_RGB2BGR)
    return array, success



if __name__ == "__main__":

    while(1):
        frame, success = get_video()
        if success:
            # show original image
            cv2.imshow('RGB image',frame)
            
            
            
            
            buttons = get_button_centers(frame)
            if buttons:
                cv2.imshow("bububijdss", buttons)
        # quit program when 'esc' key is pressed
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
    cv2.destroyAllWindows()


In [35]:
h = [[[362.5,215.5,9.6],[382.5,214.5,9.6]]]

In [34]:
h[0][0][0]

362.5

In [37]:
import math

In [39]:
h1 = [h[0][0][0], h[0][0][1]]
h2 = [h[0][1][0], h[0][1][1]]


math.dist(h2,h1)

20.024984394500787

In [ ]:
import cv2
import numpy as np
  
    
    
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

# Read image.
img = cv2.imread(image_path, cv2.IMREAD_COLOR)
  
# Convert to grayscale.
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))
  
# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 50,
               param2 = 30, minRadius = 9, maxRadius = 15)
  
print(detected_circles)
# Draw circles that are detected.
if detected_circles is not None:
  
    # Convert the circle parameters a, b and r to integers.
    detected_circles = np.uint16(np.around(detected_circles))
  
    for pt in detected_circles[0, :]:
        a, b, r = pt[0], pt[1], pt[2]
  
        # Draw the circumference of the circle.
        cv2.circle(img, (a, b), r, (0, 255, 0), 2)
  
        # Draw a small circle (of radius 1) to show the center.
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
        cv2.imshow("Detected Circle", img)
        cv2.waitKey(0)
        
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
            
    cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import math

In [ ]:
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

In [ ]:
# show the base image

image = cv2.imread(image_path)
cv2.imshow('base_image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def get_blob_params():
    params = cv2.SimpleBlobDetector_Params()

    params.filterByArea = True
    params.minArea = 150
    # params.maxArea = 250

    params.filterByCircularity = True
    params.minCircularity = 0.85
    
    return params


def get_button_centers(image):
    
    params = get_blob_params()

    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(image)
    if len(keypoints)==2:

        print(keypoints)

        print("---")
        print(get_keypoint_centers(keypoints))
        print("--------")
        print(get_angle(get_keypoint_centers(keypoints)))

        blank = np.zeros((1, 1))
        blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
        return blobs
    else:
        pass


def show_image(image, image_label):
    cv2.imshow(image_label, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def get_keypoint_centers(keypoints):
#     points_arr = []
#     for point in keypoints:
#         points_arr.append(point.pt)
    points_arr = [point.pt for point in keypoints]
    return points_arr
        
        
def get_angle(points):
    if len(points) == 2:
        delY = points[1][1] - points[0][1]
        delX = points[1][0] - points[0][0]
        # slope = delY/delX
        theta = math.atan2(delY, delX)
        theta_deg = theta*(180/np.pi)
        return theta_deg


# def verify


In [ ]:
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

In [4]:
import cv2
import numpy as np
  
    
    
images_dir = "/home/nemo/code/data/robothon_data/images/"
image_name = "frame101.jpg"

image_path = images_dir + image_name

# Read image.
img = cv2.imread(image_path, cv2.IMREAD_COLOR)
  
# Convert to grayscale.
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))
  
# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 20, param1 = 50,
               param2 = 30, minRadius = 9, maxRadius = 15)
  
print(detected_circles)
# Draw circles that are detected.
if detected_circles is not None:
  
    # Convert the circle parameters a, b and r to integers.
    detected_circles = np.uint16(np.around(detected_circles))
  
    for pt in detected_circles[0, :]:
        a, b, r = pt[0], pt[1], pt[2]
  
        # Draw the circumference of the circle.
        cv2.circle(img, (a, b), r, (0, 255, 0), 2)
  
        # Draw a small circle (of radius 1) to show the center.
        cv2.circle(img, (a, b), 1, (0, 0, 255), 3)
        cv2.imshow("Detected Circle", img)
        cv2.waitKey(0)
        
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
            
    cv2.destroyAllWindows()

[[[438.5 377.5  14.4]
  [428.5 237.5  13.2]]]


In [ ]:
#import the necessary modules
import freenect
import cv2
import numpy as np
import time

def get_blob_params():
    params = cv2.SimpleBlobDetector_Params()

    params.filterByArea = True
    params.minArea = 150
    # params.maxArea = 250

    params.filterByCircularity = True
    params.minCircularity = 0.85
    
    return params


def get_button_centers(image):
    
    params = get_blob_params()

    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(image)
    if len(keypoints)==2:

        print(keypoints)

        print("---")
        print(get_keypoint_centers(keypoints))
        print("--------")
        print(get_angle(get_keypoint_centers(keypoints)))

        blank = np.zeros((1, 1))
        blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
        return blobs
    else:
        pass


def show_image(image, image_label):
    cv2.imshow(image_label, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def get_keypoint_centers(keypoints):
#     points_arr = []
#     for point in keypoints:
#         points_arr.append(point.pt)
    points_arr = [point.pt for point in keypoints]
    return points_arr
        
        
def get_angle(points):
    if len(points) == 2:
        delY = points[1][1] - points[0][1]
        delX = points[1][0] - points[0][0]
        # slope = delY/delX
        theta = math.atan2(delY, delX)
        theta_deg = theta*(180/np.pi)
        return theta_deg


# def verify






#function to get RGB image from kinect
def get_video():
    array, success = freenect.sync_get_video()
    array = cv2.cvtColor(array,cv2.COLOR_RGB2BGR)
    return array, success

#function to get depth image from kinect
def get_depth():
    array, success = freenect.sync_get_depth()
    array = array.astype(np.uint8)
    return array, success

if __name__ == "__main__":

    while(1):
        frame, success = get_video()
        if success:
            cv2.imshow('RGB image',frame)
            
            
            buttons = get_button_centers(frame)
            if buttons:
                cv2.imshow("bububijdss", buttons)
        # quit program when 'esc' key is pressed
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
    cv2.destroyAllWindows()


In [ ]:
cap = cv2.VideoCapture("")
while(1):
    # Take each frame
    _, frame = cap.read()

    cv2.imshow('frame',frame)
    
#     buttons = get_button_centers(frame)
#     show_image(buttons, "buttons")
    
    
#     cv.imshow('mask',mask)
#     cv.imshow('res',res)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

In [ ]:
image = cv2.imread(image_path)
buttons = get_button_centers(image)
show_image(buttons, "buttons")

In [ ]:

pts = cv2.KeyPoint_convert(kp)

In [ ]:
points = [(428.8331298828125, 237.3616943359375), (449.1546936035156, 241.13426208496094)]


In [ ]:
points[0][0]

In [ ]:
delY = points[1][1] - points[0][1]
delX = points[1][0] - points[0][0]
# slope = delY/delX
theta = math.atan2(delY, delX)

In [ ]:
theta_deg = theta*(180/np.pi)

In [ ]:
print(theta)
print(theta_deg)

In [ ]:
math.atan2

In [ ]:
# detect blue button


# image = cv2.imread('blob2.jpg', 0)
# image = full_mask
image = cv2.imread(image_path)
params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.minArea = 150
# params.maxArea = 250

params.filterByCircularity = True
params.minCircularity = 0.85

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(image)


points = [p.pt for p in keypoints]
print(points)
# for point in keypoints:
#     M = cv2.moments(point)
#     print(M)

print(keypoints)


image_p = cv2.circle(image, (int(points[0][0]), int(points[0][1])), radius=0, color=(0, 0, 255), thickness=-1)

# image[int(points[0][0]), int(points[0][1])] = [0,0,255]

blank = np.zeros((1, 1))
blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Using area,circularity", blobs)
cv2.imshow("circle", image_p)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
points = [(428.8331298828125, 237.3616943359375), (449.1546936035156, 241.13426208496094)]

In [ ]:
points[0][0]

In [ ]:
#detect RED

# image = c
image = cv2.imread(image_path)
result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
 
# lower boundary RED color range values; Hue (0 - 10)
lower1 = np.array([0, 100, 20])
upper1 = np.array([10, 255, 255])
 
# upper boundary RED color range values; Hue (160 - 180)
lower2 = np.array([160,100,20])
upper2 = np.array([179,255,255])

 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)



params.filterByArea = True
params.minArea = 150

params.filterByCircularity = True
params.minCircularity = 0.8

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(image)

print(keypoints)

blank = np.zeros((1, 1))
blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)





cv2.imshow('mask', full_mask)
cv2.imshow('result', result)
cv2.imshow("Using area,circularity", blobs)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
params.filterByArea = True
params.minArea = 150

params.filterByCircularity = True
params.minCircularity = 0.8

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(result)

print(keypoints)

blank = np.zeros((1, 1))
blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


In [ ]:
cv2.imshow('mask', full_mask)

In [ ]:

# image = cv2.imread('blob2.jpg', 0)
# image = full_mask
image = cv2.imread(image_path)
params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.minArea = 150

params.filterByCircularity = True
params.minCircularity = 0.8

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(image)

print(keypoints)

blank = np.zeros((1, 1))
blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 0, 255),cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Using area,circularity", blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# find blob centers

# read image through command line
img = cv2.imread(image_path)

# convert the image to grayscale
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert the grayscale image to binary image
ret,thresh = cv2.threshold(gray_image,127,255,0)

# find contours in the binary image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

print(len(contours))

for c in contours:
    
    # calculate moments for each contour
    M = cv2.moments(c)

    # calculate x,y coordinate of center
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        print("pri")
        cX, cY = 0, 0
        
    cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
    cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # display the image
    cv2.imshow("Image", img)
    cv2.waitKey(0)

    
    
# if M["m00"] != 0:
#  cX = int(M["m10"] / M["m00"])
#  cY = int(M["m01"] / M["m00"])
# else:
#  cX, cY = 0, 0

    

In [ ]:
# import cv2 as cv
# import numpy as np
# cap = cv.VideoCapture(0)
# while(1):
#     # Take each frame
#     _, frame = cap.read()
#     # Convert BGR to HSV
#     hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
#     # define range of blue color in HSV
#     lower_blue = np.array([110,50,50])
#     upper_blue = np.array([130,255,255])
#     # Threshold the HSV image to get only blue colors
#     mask = cv.inRange(hsv, lower_blue, upper_blue)
#     # Bitwise-AND mask and original image
#     res = cv.bitwise_and(frame,frame, mask= mask)
#     cv.imshow('frame',frame)
#     cv.imshow('mask',mask)
#     cv.imshow('res',res)
#     k = cv.waitKey(5) & 0xFF
#     if k == 27:
#         break
# cv.destroyAllWindows()

In [ ]:

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(frame,frame, mask= mask)
cv2.imshow('frame',frame)
cv2.imshow('mask',mask)
cv2.imshow('res',res)

 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
 
# # lower boundary RED color range values; Hue (0 - 10)
# lower1 = np.array([0, 100, 20])
# upper1 = np.array([10, 255, 255])
 
# # upper boundary RED color range values; Hue (160 - 180)
# lower2 = np.array([160,100,20])
# upper2 = np.array([179,255,255])
 
    
# lower_mask = cv2.inRange(image, lower1, upper1)
# upper_mask = cv2.inRange(image, lower2, upper2)
 
# full_mask = lower_mask + upper_mask;

# define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])
# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower_blue, upper_blue)
# Bitwise-AND mask and original image
res = cv2.bitwise_and(frame,frame, mask= mask)
# 
# result = cv2.bitwise_and(result, result, mask=full_mask)
 
cv2.imshow('mask', mask)
cv2.imshow('result', res)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#detect RED

result = image.copy()
 
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
 
# lower boundary RED color range values; Hue (0 - 10)
lower1 = np.array([0, 100, 20])
upper1 = np.array([10, 255, 255])
 
# upper boundary RED color range values; Hue (160 - 180)
lower2 = np.array([160,100,20])
upper2 = np.array([179,255,255])
 
lower_mask = cv2.inRange(image, lower1, upper1)
upper_mask = cv2.inRange(image, lower2, upper2)
 
full_mask = lower_mask + upper_mask;
 
result = cv2.bitwise_and(result, result, mask=full_mask)
 
cv2.imshow('mask', full_mask)
cv2.imshow('result', result)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

In [ ]:
## mask of red color
mask1 = cv2.inRange(img, (0, 0, 50), (50, 50,255))

## mask of blue color
mask2 = cv2.inRange(img, (50,0,0), (255, 50, 50))

cv2.imshow('Original Image',img)
cv2.imshow('mask red color',mask1)
cv2.imshow('mask blue color',mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
 

In [ ]:
image = cv2.imread('rose.jpg')
cv2.imshow("Original", image)
 